<a href="https://colab.research.google.com/github/Sheryar-bit/Lang-Chain/blob/main/Img_To_Txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers torch pillow

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

Loafing model and transformer needed( Using bit slow models for now  )

In [ ]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Check if GPU available or not if not then use CPU
model.to(device)

In [ ]:
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

Create captioning function in Python

In [ ]:
#takes in a list of image file paths and generates text captions for them using a vision-language model.
def predict_step(image_paths):
  images = []

  ## will Loop over each image path provided
  for image_path in image_paths:
    i_image = Image.open(image_path)
    #Optional (RGB color)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    #Add the processed image to the list
    images.append(i_image)

  # This block takes the preprocessed image tensors, feeds them into the vision-language model to
  # generate captions, decodes the output token IDs into human-readable text, cleans up whitespace,

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values  # Extracting pixel values (numerical tensors) from images using the feature extractor
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  #  Strip leading/trailing whitespace from each prediction
  preds = [pred.strip() for pred in preds]
  return preds

In [ ]:
from google.colab import drive
drive.mount('') ##Any Image text will ge genrated